In [14]:
import pandas as pd

# Load the input files using Pandas
item_list_df = pd.read_excel('/Users/bfillm/Desktop/allocation testing/item_list.xlsx')
price_list_df = pd.read_excel('/Users/bfillm/Desktop/allocation testing/price_list.xlsx')
factory_demand_df = pd.read_excel('/Users/bfillm/Desktop/allocation testing//factory_demand.xlsx')
target_volume_df = pd.read_excel('/Users/bfillm/Desktop/allocation testing/target_volume.xlsx')

In [15]:
# Calculate MIN TARGET VOLUME and MAX TARGET VOLUME
target_volume_df['Min Target Volume Per Group Type'] = (target_volume_df['Target %'] - target_volume_df['Target % Range']) * target_volume_df['Total Volume']
target_volume_df['Max Target Volume Per Group Type'] = (target_volume_df['Target %'] + target_volume_df['Target % Range']) * target_volume_df['Total Volume']

target_volume_df.head()

,CSO Cleaned Supplier Name,CSO Country,Target %,Target % Range,Min Target Volume Per Group Type,Max Target Volume Per Group Type,Supplier Type,Total Volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASIATAN,CHINA,0.02,0.03,-738840.48,3694202.40,Leather,73884048.0
2,ASIATAN,VIETNAM,0.03,0.03,0.00,4433042.88,Leather,73884048.0
3,DAEHWA INDONESIA PT,INDONESIA,0.06,0.02,2955361.92,5910723.84,Leather,73884048.0
4,ECCO LEATHER B.V.,INDONESIA,0.07,0.03,2955361.92,7388404.80,Leather,73884048.0


In [16]:
price_list_df.rename(columns={"Company":"SUPPLIER", "[new_bid_price_fob_origin_bid]":"BID PRICE"}, inplace=True)
item_list_df.rename(columns={"CSO Cleaned Supplier Country":"SUPPLIER COUNTRY", "CSO Cleaned Supplier Name":"SUPPLIER"}, inplace=True)

In [17]:
price_list_df.head(5)

,Bid Round,Item Type,Item State,Item Name,SUPPLIER,Supplier Name,Bid ID,Delete bid (applied in Modified Bids upload),Standing Bid,Received,BID PRICE,Supplier Country,PCX_SPL_MTL_ID,PDM SPL LOC NAME,PDM_SPL_MTL_NUM
0,INITIAL PRICE UPLOAD,material,Active,Item-0001,HONG WON,"Nike, Moussa Zaatar",9220548391120216045,NaN,T,2022-04-26 10:14:24,0.0398,VIETNAM,445738.0,HONG WON (VIETNAM),254365.584.2
1,INITIAL PRICE UPLOAD,material,Active,Item-0002,I HSING WEAVING WEBBING CO LTD,"Nike, Moussa Zaatar",9220548391120232760,NaN,T,2022-04-26 10:14:32,0.0336,CHINA,457174.0,I HSING WEAVING WEBBING CO LTD (CHINA),254365.41.2
2,INITIAL PRICE UPLOAD,material,Active,Item-0003,PAIHO GROUP,"Nike, Moussa Zaatar",9220548391120233873,NaN,T,2022-04-26 10:14:33,0.0310,CHINA,457175.0,PAIHO - UPPER COMPONENTS (CHINA),254365.6.3
3,INITIAL PRICE UPLOAD,material,Active,Item-0004,PAIHO GROUP,"Nike, Moussa Zaatar",9220548391120233879,NaN,T,2022-04-26 10:14:33,0.0417,INDONESIA,447119.0,PAIHO - UPPER COMPONENTS (INDONESIA),254365.6.5
4,INITIAL PRICE UPLOAD,material,Active,Item-0005,PAIHO GROUP,"Nike, Moussa Zaatar",9220548391120233885,NaN,T,2022-04-26 10:14:33,0.0370,VIETNAM,447128.0,PAIHO - UPPER COMPONENTS (VIETNAM),254365.6.2


In [18]:
# Merge ITEM LIST with PRICE LIST on 'PDM MTL NUM'
merged_df = pd.merge(item_list_df, price_list_df, on='PCX_SPL_MTL_ID', how='inner')

# Prioritize by lower price
merged_df = merged_df.sort_values(by='BID PRICE')

In [22]:
# Apply LOCAL FOR LOCAL logic (matching supplier and factory country)
merged_df['LOCAL_FOR_LOCAL'] = merged_df['SUPPLIER COUNTRY'] == merged_df['Supplier Country']
merged_df = merged_df.sort_values(by=['BID PRICE', 'LOCAL_FOR_LOCAL'], ascending=[True, False])

merged_df.head()

,Name,Delete,Visibility (State),PDM MTL NUM,Factory Code,PDM SPL MTL NAME,PDM SPL MTL NUM,PCX_SPL_MTL_ID,PCX Mtl ID,Delivery Term,...,Supplier Name,Bid ID,Delete bid (applied in Modified Bids upload),Standing Bid,Received,BID PRICE,Supplier Country,PDM SPL LOC NAME_y,PDM_SPL_MTL_NUM_y,LOCAL_FOR_LOCAL
5703,Item-4876,NaN,Active,238587,NaN,"(PM) HIGH DENSITY, SMOOTH GRAIN, 1.4MM, DE-126 SM",NaN,NaN,NaN,NaN,...,"Nike, Moussa Zaatar",9220548391120222613,NaN,T,2022-11-16 13:57:37,0.0,VIETNAM,DAEWON (VIETNAM),238587.26.2,True
5771,Item-4964,NaN,Active,232858,NaN,"(PM) PIGMENTED NAPPA, 1.4-1.6MM, MATTE",NaN,NaN,169616.0,NaN,...,"Nike, Moussa Zaatar",9220548391120237453,NaN,T,2022-11-16 13:57:42,0.0,VIETNAM,ISA INDUSTRIAL CO LTD (VIETNAM),229418.31.2,True
5774,Item-4964,NaN,Active,232858,NaN,"(PM) PIGMENTED NAPPA, 1.4-1.6MM, MATTE",NaN,NaN,169616.0,NaN,...,"Nike, Moussa Zaatar",9220548391120237458,NaN,T,2022-11-16 13:57:42,0.0,VIETNAM,ISA INDUSTRIAL CO LTD (VIETNAM),229425.31.2,True
5775,Item-4964,NaN,Active,232858,NaN,"(PM) PIGMENTED NAPPA, 1.4-1.6MM, MATTE",NaN,NaN,169616.0,NaN,...,"Nike, Moussa Zaatar",9220548391120237463,NaN,T,2022-11-16 13:57:42,0.0,VIETNAM,ISA INDUSTRIAL CO LTD (VIETNAM),229432.31.2,True
5780,Item-4964,NaN,Active,232858,NaN,"(PM) PIGMENTED NAPPA, 1.4-1.6MM, MATTE",NaN,NaN,169616.0,NaN,...,"Nike, Moussa Zaatar",9220548391120237473,NaN,T,2022-11-16 13:57:42,0.0,VIETNAM,ISA INDUSTRIAL CO LTD (VIETNAM),232831.31.2,True


In [23]:
target_volume_df.head(5)

,CSO Cleaned Supplier Name,CSO Country,Target %,Target % Range,Min Target Volume Per Group Type,Max Target Volume Per Group Type,Supplier Type,Total Volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASIATAN,CHINA,0.02,0.03,-738840.48,3694202.40,Leather,73884048.0
2,ASIATAN,VIETNAM,0.03,0.03,0.00,4433042.88,Leather,73884048.0
3,DAEHWA INDONESIA PT,INDONESIA,0.06,0.02,2955361.92,5910723.84,Leather,73884048.0
4,ECCO LEATHER B.V.,INDONESIA,0.07,0.03,2955361.92,7388404.80,Leather,73884048.0


In [ ]:
import pandas as pd

# Load the input files using Pandas
item_list_df = pd.read_excel('ITEM LIST.xlsx')
price_list_df = pd.read_excel('PRICE LIST.xlsx')
factory_demand_df = pd.read_excel('FACTORY DEMAND.xlsx')
target_volume_df = pd.read_excel('TARGET VOLUME.xlsx')

# Print data types to check if any conversion is needed
print("Item List Data Types:\n", item_list_df.dtypes)
print("Price List Data Types:\n", price_list_df.dtypes)
print("Factory Demand Data Types:\n", factory_demand_df.dtypes)
print("Target Volume Data Types:\n", target_volume_df.dtypes)

# Convert columns to appropriate data types if needed
item_list_df['PCX SPL MTL ID'] = item_list_df['PCX SPL MTL ID'].astype(str).fillna(item_list_df['PCX MTL ID'])
price_list_df['PCX SPL MTL ID'] = price_list_df['PCX SPL MTL ID'].astype(str).fillna(price_list_df['PCX MTL ID'])
factory_demand_df['PCX SPL MTL ID'] = factory_demand_df['PCX SPL MTL ID'].astype(str).fillna(factory_demand_df['PCX MTL ID'])

# Handle any potential conversion issues
factory_demand_df['VOLUME'] = pd.to_numeric(factory_demand_df['VOLUME'], errors='coerce').fillna(0).astype(int)
price_list_df['BID PRICE'] = pd.to_numeric(price_list_df['BID PRICE'], errors='coerce').fillna(0).astype(float)
target_volume_df['TOTAL VOLUME'] = pd.to_numeric(target_volume_df['TOTAL VOLUME'], errors='coerce').fillna(0).astype(int)
target_volume_df['TARGET %'] = pd.to_numeric(target_volume_df['TARGET %'], errors='coerce').fillna(0).astype(float)
target_volume_df['TARGET % RANGE'] = pd.to_numeric(target_volume_df['TARGET % RANGE'], errors='coerce').fillna(0).astype(float)

# Calculate MIN TARGET VOLUME and MAX TARGET VOLUME
target_volume_df['MIN TARGET VOLUME'] = (target_volume_df['TARGET %'] - target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']
target_volume_df['MAX TARGET VOLUME'] = (target_volume_df['TARGET %'] + target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']

print("Data Preprocessing Complete")

In [ ]:
# Merge ITEM LIST with PRICE LIST on 'PCX SPL MTL ID'
merged_df = pd.merge(item_list_df, price_list_df, on='PCX SPL MTL ID', how='inner')

# Assign preference by LOCAL FOR LOCAL and BID PRICE
merged_df['LOCAL_FOR_LOCAL'] = merged_df['SUPPLIER_x'] == merged_df['SUPPLIER COUNTRY_y']
merged_df = merged_df.sort_values(by=['LOCAL_FOR_LOCAL', 'BID PRICE'], ascending=[False, True])

merged_df.head()

In [ ]:
def allocate_volume(merged_df, target_volume_df, factory_demand_df):
    allocations = []
    
    for _, supplier in target_volume_df.iterrows():
        supplier_name = supplier['SUPPLIER']
        min_volume = supplier['MIN TARGET VOLUME']
        max_volume = supplier['MAX TARGET VOLUME']
        supplier_df = merged_df[merged_df['SUPPLIER_x'] == supplier_name]
        
        allocated_volume = 0
        for _, row in supplier_df.iterrows():
            if allocated_volume >= max_volume:
                break

            factory_options = factory_demand_df[factory_demand_df['PCX SPL MTL ID'] == row['PCX SPL MTL ID']]
            for _, factory in factory_options.iterrows():
                if allocated_volume >= max_volume:
                    break
                if factory['VOLUME'] > 0:
                    volume_to_allocate = min(max_volume - allocated_volume, factory['VOLUME'])
                    allocations.append({
                        'PCX SPL MTL ID': row['PCX SPL MTL ID'],
                        'SUPPLIER': row['SUPPLIER_x'],
                        'FACTORY CODE': factory['FACTORY CODE'],
                        'ALLOCATION (UNITS)': volume_to_allocate,
                        'PRICE': row['BID PRICE']
                    })
                    allocated_volume += volume_to_allocate
                    factory_demand_df.loc[factory_demand_df['PCX SPL MTL ID'] == factory['PCX SPL MTL ID'], 'VOLUME'] -= volume_to_allocate
    
    return pd.DataFrame(allocations)

# Perform the allocation
allocation_df = allocate_volume(merged_df, target_volume_df, factory_demand_df)

allocation_df.head()

In [29]:
def allocate_volume(merged_df, target_volume_df, factory_demand_df):
    allocations = []
    
    for _, supplier in target_volume_df.iterrows():
        supplier_name = supplier['SUPPLIER']
        min_volume = supplier['MIN TARGET VOLUME']
        max_volume = supplier['MAX TARGET VOLUME']
        supplier_df = merged_df[merged_df['SUPPLIER_x'] == supplier_name]
        
        allocated_volume = 0
        for _, row in supplier_df.iterrows():
            if allocated_volume >= max_volume:
                break

            factory_options = factory_demand_df[factory_demand_df['PCX SPL MTL ID'] == row['PCX SPL MTL ID']]
            for _, factory in factory_options.iterrows():
                if allocated_volume >= max_volume:
                    break
                if factory['VOLUME'] > 0:
                    volume_to_allocate = min(max_volume - allocated_volume, factory['VOLUME'])
                    allocations.append({
                        'PCX SPL MTL ID': row['PCX SPL MTL ID'],
                        'SUPPLIER': row['SUPPLIER_x'],
                        'FACTORY CODE': factory['FACTORY CODE'],
                        'ALLOCATION (UNITS)': volume_to_allocate,
                        'PRICE': row['BID PRICE']
                    })
                    allocated_volume += volume_to_allocate
                    factory_demand_df.loc[factory_demand_df['PCX SPL MTL ID'] == factory['PCX SPL MTL ID'], 'VOLUME'] -= volume_to_allocate
    
    return pd.DataFrame(allocations)

# Perform the allocation
allocation_df = allocate_volume(merged_df, target_volume_df, factory_demand_df)

allocation_df.head()

In [ ]:
# Output the allocation results in the same format as ALLOCATION RESULT file
allocation_result_columns = ['PCX SPL MTL ID', 'SUPPLIER', 'FACTORY CODE', 'ALLOCATION (UNITS)', 'PRICE']
allocation_df = allocation_df[allocation_result_columns]

# Save the results to an Excel file
allocation_df.to_excel('ALLOCATION_RESULT.xlsx', index=False)

print("Allocation results saved to ALLOCATION_RESULT.xlsx")